In [1]:
# this file is implementation of a simple neural network on the data file

In [2]:
import pandas as pd

# read data
data = pd.read_csv('data.csv')
data.head()

,acc_pitch,acc_roll,acc_yaw,gyro_pitch,gyro_roll,gyro_yaw,magno_pitch,magno_roll,magno_yaw,y
0,0.15,0.29,-45.0,-0.1,-0.1,-0.1,21.15,9.07,-42.95,2
1,0.15,0.29,-45.0,-0.1,-0.1,-0.1,21.16,9.07,-42.95,2
2,0.29,0.29,-45.0,-0.1,-0.1,-0.1,21.15,9.09,-42.95,2
3,0.29,0.29,-45.0,-0.1,-0.1,-0.1,21.15,9.09,-42.95,2
4,-0.44,-0.00,-45.0,-0.0,-0.0,45.0,8.15,9.10,-44.70,2


In [6]:
# split data into X and y
X = data.iloc[:, 0:9]
y = data.iloc[:, 9]

In [8]:
# split data into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
import numpy as np
import tensorflow as tf

# Create a simple neural network with leaky ReLU activation
model_nn = tf.keras.Sequential([
    tf.keras.layers.Dense(8, input_shape=(9,), activation=tf.nn.leaky_relu),
    tf.keras.layers.Dense(8, activation=tf.nn.leaky_relu),
    tf.keras.layers.Dense(8, activation=tf.nn.leaky_relu),
    tf.keras.layers.Dense(8, activation=tf.nn.leaky_relu),
    tf.keras.layers.Dense(4, activation='softmax')
])

# Compile the model
model_nn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with your dataset
# Assuming X_train is your input features and y_train is your labels
model_nn.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model_nn.evaluate(X_test, y_test, batch_size=32)

Epoch 1/10
49/49 [==============================] - 2s 13ms/step - loss: 1.1842 - accuracy: 0.5210
Epoch 2/10
49/49 [==============================] - 0s 8ms/step - loss: 0.8149 - accuracy: 0.5712
Epoch 3/10
49/49 [==============================] - 0s 8ms/step - loss: 0.7470 - accuracy: 0.5725
Epoch 4/10
49/49 [==============================] - 1s 13ms/step - loss: 0.7134 - accuracy: 0.5738
Epoch 5/10
49/49 [==============================] - 1s 11ms/step - loss: 0.6939 - accuracy: 0.5725
Epoch 6/10
49/49 [==============================] - 0s 10ms/step - loss: 0.6740 - accuracy: 0.5887
Epoch 7/10
49/49 [==============================] - 0s 10ms/step - loss: 0.6655 - accuracy: 0.5874
Epoch 8/10
49/49 [==============================] - 0s 9ms/step - loss: 0.6672 - accuracy: 0.5822
Epoch 9/10
49/49 [==============================] - 0s 9ms/step - loss: 0.6654 - accuracy: 0.5874
Epoch 10/10
49/49 [==============================] - 0s 9ms/step - loss: 0.7129 - accuracy: 0.5622


In [33]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Assuming your data is in a DataFrame called 'data_df'
# Features and labels
X = data.drop('y', axis=1).values
y = data['y'].values

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Adjust target labels to start from 0
y_train -= 1
y_test -= 1

# PyTorch data loaders
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define the PyTorch neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(9, 8)
        self.fc2 = nn.Linear(8, 8)
        self.fc3 = nn.Linear(8, 8)
        self.fc4 = nn.Linear(8, 8)
        self.fc5 = nn.Linear(8, 4)

    def forward(self, x):
        x = nn.functional.leaky_relu(self.fc1(x))
        x = nn.functional.leaky_relu(self.fc2(x))
        x = nn.functional.leaky_relu(self.fc3(x))
        x = nn.functional.leaky_relu(self.fc4(x))
        x = nn.functional.softmax(self.fc5(x), dim=1)
        return x

# Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the PyTorch model
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(torch.tensor(X_test, dtype=torch.float32))
    _, predicted = torch.max(test_outputs, 1)
    test_accuracy = accuracy_score(y_test, predicted.numpy())
    print('Test accuracy:', test_accuracy)


Test accuracy: 0.788659793814433


# model retraining

In [36]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Assuming your data is in a DataFrame called 'data_df'
# Features and labels
X = data.drop('y', axis=1).values
y = data['y'].values

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to save model weights
def save_model_weights(model, filepath):
    torch.save(model.state_dict(), filepath)

# Function to load model weights
def load_model_weights(model, filepath):
    model.load_state_dict(torch.load(filepath))
    model.eval()

# Function to retrain the model using specified weights
def retrain_model(model, X_train, y_train, num_epochs=10):
    train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train-1))
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

# Function to calculate model accuracy
def calculate_accuracy(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        test_outputs = model(torch.tensor(X_test, dtype=torch.float32))
        _, predicted = torch.max(test_outputs, 1)
        test_accuracy = accuracy_score(y_test, predicted.numpy())
        return test_accuracy

# Define the PyTorch neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(9, 8)
        self.fc2 = nn.Linear(8, 8)
        self.fc3 = nn.Linear(8, 8)
        self.fc4 = nn.Linear(8, 8)
        self.fc5 = nn.Linear(8, 4)

    def forward(self, x):
        x = nn.functional.leaky_relu(self.fc1(x))
        x = nn.functional.leaky_relu(self.fc2(x))
        x = nn.functional.leaky_relu(self.fc3(x))
        x = nn.functional.leaky_relu(self.fc4(x))
        x = nn.functional.softmax(self.fc5(x), dim=1)
        return x

# Initialize the model
model = SimpleNN()

# Save the initial model weights
save_model_weights(model, 'initial_model_weights.pth')

# Train the model with initial weights
retrain_model(model, X_train, y_train)

# Calculate and print the initial model accuracy
initial_accuracy = calculate_accuracy(model, X_test, y_test)
print('Initial model accuracy:', initial_accuracy)

# Load the initial model weights and retrain the model
load_model_weights(model, 'initial_model_weights.pth')
retrain_model(model, X_train, y_train, num_epochs=5)

# Calculate and print the retrained model accuracy
retrained_accuracy = calculate_accuracy(model, X_test, y_test)
print('Retrained model accuracy:', retrained_accuracy)

Initial model accuracy: 0.10051546391752578
Retrained model accuracy: 0.0979381443298969
